##paper stats generator
ryoung@unlv.edu


In [1]:
# Cell 1

import os
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, cohen_kappa_score, mean_absolute_error

# (Optional) For fuzzy matching
# !pip install rapidfuzz
# from rapidfuzz import process

# Ensure output directory exists
output_dir = '03_data_ai'
os.makedirs(output_dir, exist_ok=True)

# Read the parquet file into a DataFrame
df_stats = pd.read_parquet(f'../{output_dir}/part_3_covariates_final.parquet')

print("DataFrame loaded. Shape:", df_stats.shape)
df_stats.head(3)

DataFrame loaded. Shape: (64, 68)


,Index,NCTId,LeadSponsorClass,LeadSponsorName,Condition,OfficialTitle,BriefTitle,Acronym,StudyType,InterventionType,...,relevant_quotes_anthropic_claude_3.5_sonnet,brain_stimulation_used_meta_llama_llama_3_70b_instruct,stimulation_details_primary_type_meta_llama_llama_3_70b_instruct,stimulation_details_is_noninvasive_meta_llama_llama_3_70b_instruct,stimulation_details_primary_target_meta_llama_llama_3_70b_instruct,stimulation_details_secondary_targets_meta_llama_llama_3_70b_instruct,stimulation_details_parameters_intensity_meta_llama_llama_3_70b_instruct,stimulation_details_parameters_duration_meta_llama_llama_3_70b_instruct,confidence_level_meta_llama_llama_3_70b_instruct,relevant_quotes_meta_llama_llama_3_70b_instruct
0,None,NCT06145776,OTHER,Federal University of Paraíba,"Parkinson Disease, Cognition Disorder, EEG Wit...",Effects of Cognitive-motor Dual-Task Training ...,The Role of EEG in Identifying Cognitive Chang...,,INTERVENTIONAL,",",...,"[""It is possible that strategies such as Trans...",Yes,tDCS,True,,[],,,High,"[""Transcranial Direct Current Stimulation can ..."
1,None,NCT04174573,OTHER,Maharishi Markendeswar University (Deemed to b...,Parkinson Disease,Efficacy of Group Therapy With Transcranial Di...,Group Therapy With Transcranial Direct Current...,GTBQ-PD,INTERVENTIONAL,",",...,"[""Transcranial direct current stimulation (tDC...",Yes,tDCS,True,Primary motor cortex M1,[],,20 minutes,High,"[""Anodal tDCS can increase M1 excitability, re..."
2,None,NCT03191916,OTHER,Sanford Health,Parkinson Disease,Transcranial Direct Current Stimulation (tDCS)...,Transcranial Direct Current Stimulation for Co...,tDCS,INTERVENTIONAL,",",...,"[""Transcranial direct current stimulation (tDC...",Yes,tDCS,True,LDLPFC,[],2 milliamps (mA),20 minutes,High,"[""During tDCS, low-voltage, low amplitude curr..."


---- mroe ---


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, cohen_kappa_score

# 1) Configure matplotlib to use Times New Roman globally
plt.rcParams["font.family"] = "Times New Roman"

def compute_binary_agreement(df: pd.DataFrame, col_a: str, col_b: str) -> dict:
    """
    Computes confusion matrix, observed agreement, expected agreement,
    and Cohen's Kappa for two binary (0/1) columns.
    """
    a = df[col_a]
    b = df[col_b]

    # confusion_matrix(labels=[0,1]) => [[TN, FP], [FN, TP]]
    cm = confusion_matrix(a, b, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()
    total = tn + fp + fn + tp

    # Observed agreement
    p_o = (tn + tp) / total if total else 0

    # Expected agreement
    row1_sum = tp + fn  # total '1' in col A
    row2_sum = fp + tn  # total '0' in col A
    col1_sum = tp + fp  # total '1' in col B
    col2_sum = fn + tn  # total '0' in col B
    if total == 0:
        p_e = 0
    else:
        p_e = (row1_sum * col1_sum + row2_sum * col2_sum) / (total**2)

    # Sklearn kappa
    kappa_sklearn = cohen_kappa_score(a, b)

    return {
        "ConfusionMatrix (tn, fp, fn, tp)": (tn, fp, fn, tp),
        "ObservedAgreement": p_o,
        "ExpectedAgreement": p_e,
        "CohensKappa": kappa_sklearn
    }
def df_to_png(df: pd.DataFrame, filename: str = "comparison_table.png", dpi=150, title=None):
    """
    Renders a DataFrame as a table in Matplotlib and saves as a PNG.
    Uses Times New Roman font. Minimizes extra whitespace by using
    constrained_layout and careful scaling.
    """
    # Smaller figure size; constrained_layout tries to handle spacing automatically
    fig, ax = plt.subplots(figsize=(8, 2), constrained_layout=True)
    ax.axis("off")  # Hide the x/y axes

    if title:
        # Slightly reduce pad
        plt.title(title, fontsize=12, pad=5)

    # Convert df to a list of lists (cell text) + list of column names
    table_data = [df.columns.tolist()] + df.values.tolist()
    # Create the table in the center
    tab = ax.table(cellText=table_data, loc="center")

    # Set a uniform font size
    tab.set_fontsize(11)

    # Remove or reduce table.scale if you want even tighter spacing:
    # tab.scale(1.0, 1.0)  # 1.0,1.0 means no scaling beyond default

    # Optionally, if you need columns 1..4 narrower:
    for (row, col), cell in tab.get_celld().items():
        if col in [1, 2, 3, 4]:
            cell.set_width(0.3)
        else:
            cell.set_width(1.0)
        # Center-align text
        cell.get_text().set_horizontalalignment("center")
        cell.get_text().set_verticalalignment("center")

    # We do NOT call plt.tight_layout() since constrained_layout=True is used.
    plt.savefig(filename, dpi=dpi, bbox_inches="tight")
    plt.close(fig)
    print(f"Table saved to {filename}")

########################################
# 2) Load your DataFrame
########################################
output_dir = "03_data_ai"
os.makedirs(output_dir, exist_ok=True)

df_stats = pd.read_parquet(f"../{output_dir}/part_3_covariates_final.parquet")
print("DataFrame loaded. Shape:", df_stats.shape)

########################################
# 3) Convert 'Yes'/'No' to 1/0
########################################
brain_stim_cols = [
    "brain_stimulation_used_gpt_4",
    "brain_stimulation_used_anthropic_claude_3.5_sonnet",
    "brain_stimulation_used_meta_llama_llama_3_70b_instruct"
]

for col in brain_stim_cols:
    df_stats[col] = df_stats[col].map({"Yes": 1, "No": 0})

df_stats.dropna(subset=brain_stim_cols, inplace=True)

for col in brain_stim_cols:
    df_stats[col] = df_stats[col].astype(int)

print("After converting Yes/No -> 1/0, dataframe shape:", df_stats.shape)
print(df_stats[brain_stim_cols].head(5))

########################################
# 4) Compare each pair
########################################
pairs_to_compare = [
    ("brain_stimulation_used_gpt_4",
     "brain_stimulation_used_anthropic_claude_3.5_sonnet",
     "GPT-4 vs Claude"),

    ("brain_stimulation_used_gpt_4",
     "brain_stimulation_used_meta_llama_llama_3_70b_instruct",
     "GPT-4 vs Llama"),

    ("brain_stimulation_used_anthropic_claude_3.5_sonnet",
     "brain_stimulation_used_meta_llama_llama_3_70b_instruct",
     "Claude vs Llama")
]

results_list = []

for col_a, col_b, label in pairs_to_compare:
    res = compute_binary_agreement(df_stats, col_a, col_b)
    tn, fp, fn, tp = res["ConfusionMatrix (tn, fp, fn, tp)"]
    row_dict = {
        "Comparison": label,
        "TN": tn,
        "FP": fp,
        "FN": fn,
        "TP": tp,
        "ObservedAgreement": f"{res['ObservedAgreement']:.3f}",
        "ExpectedAgreement": f"{res['ExpectedAgreement']:.3f}",
        "Cohen's Kappa": f"{res['CohensKappa']:.3f}",
    }
    results_list.append(row_dict)

df_compare = pd.DataFrame(results_list)

########################################
# 5) Print & Save Table
########################################
print("\nPairwise Comparison of Brain Stimulation Columns:\n")
print(df_compare.to_string(index=False))

df_to_png(
    df_compare,
    filename="brain_stimulation_all_pairs.png",
    dpi=150,
    title="Comparison of Brain Stimulation (GPT-4, Claude, Llama)"
)

DataFrame loaded. Shape: (64, 68)
After converting Yes/No -> 1/0, dataframe shape: (63, 68)
   brain_stimulation_used_gpt_4  \
0                             1   
1                             1   
2                             1   
3                             1   
4                             1   

   brain_stimulation_used_anthropic_claude_3.5_sonnet  \
0                                                  1    
1                                                  1    
2                                                  1    
3                                                  1    
4                                                  1    

   brain_stimulation_used_meta_llama_llama_3_70b_instruct  
0                                                  1       
1                                                  1       
2                                                  1       
3                                                  1       
4                                                  1  

/var/folders/yt/5983d8ds4bzdkdnzv7nz9qk40000gn/T/ipykernel_63980/1027655927.py:81: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or Axes decorations smaller.
  plt.savefig(filename, dpi=dpi, bbox_inches="tight")


In [3]:
import pandas as pd

print(pd.__version__)

2.2.3


In [4]:
import dataframe_image as dfi

# Suppose df_compare is your final DataFrame with columns:
# [Comparison, TN, FP, FN, TP, ObservedAgreement, ExpectedAgreement, Cohen's Kappa]

df_style = (
    df_compare.style
    .set_properties(**{
        "text-align": "center",
        "font-family": "Times New Roman",
        "font-size": "12px"
    })
    .set_table_styles([
        # Style for column headers
        dict(selector="thead th",
             props=[
                 ("border", "1px solid black"),
                 ("background-color", "#f2f2f2"),
                 ("font-family", "Times New Roman"),
                 ("font-size", "12px")
             ]),
        # Style for table body cells
        dict(selector="tbody td",
             props=[("border", "1px solid black")]),
        # Hide the index cells (older trick for older pandas versions)
        dict(selector=".row_heading, .blank",
             props=[("display", "none")])
    ])
)

dfi.export(df_style, "brain_stimulation_all_pairs.png", dpi=300, max_rows=None, max_cols=None)
print("Saved high-resolution table to brain_stimulation_all_pairs.png")
# Finally, export to PNG
# (If you prefer a higher resolution, increase dpi, e.g., dpi=300)
# dfi.export(df_style, "brain_stimulation_all_pairs.png")

print("Saved table to brain_stimulation_all_pairs.png")

Saved high-resolution table to brain_stimulation_all_pairs.png
Saved table to brain_stimulation_all_pairs.png
